In [34]:
from konlpy.tag import Mecab 

mec = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [37]:
import pandas as pd
import os

df = pd.read_csv("../../../outputs/week1/df3_2_artsCleanEtc.csv")


In [38]:
mec.pos('10년차')

[('10', 'SN'), ('년', 'NNBC'), ('차', 'NNG')]

In [39]:

#초기 구조화: 필수 -> 이후 case로 분기
series_mec = df['art_title'].map(lambda x : mec.pos(x, flatten=False, join=False))

df_mec = pd.DataFrame(columns=['art_id','token'])

for i in range(len(series_mec)):
    print(f'level {i}')
    for j in range(len(series_mec[i])):
        df_mec = df_mec.append({'art_id':i, 'token': series_mec[i][j]}, ignore_index=True)
        
# # 저장주의!!!
# df_mec.to_csv('df_mec_title_all_2.csv',encoding='utf-8-sig', index=False)


level 0
level 1
level 2
level 3
level 4
level 5
level 6
level 7
level 8
level 9
level 10
level 11
level 12
level 13
level 14
level 15
level 16
level 17
level 18
level 19
level 20
level 21
level 22
level 23
level 24
level 25
level 26
level 27
level 28
level 29
level 30
level 31
level 32
level 33
level 34
level 35
level 36
level 37
level 38
level 39
level 40
level 41
level 42
level 43
level 44
level 45
level 46
level 47
level 48
level 49
level 50
level 51
level 52
level 53
level 54
level 55
level 56
level 57
level 58
level 59
level 60
level 61
level 62
level 63
level 64
level 65
level 66
level 67
level 68
level 69
level 70
level 71
level 72
level 73
level 74
level 75
level 76
level 77
level 78
level 79
level 80
level 81
level 82
level 83
level 84
level 85
level 86
level 87
level 88
level 89
level 90
level 91
level 92
level 93
level 94
level 95
level 96
level 97
level 98
level 99
level 100
level 101
level 102
level 103
level 104
level 105
level 106
level 107
level 108
level 109
level 110


### -> outputs/week3/df_mec_title_all.csv 저장됨
<br>

## Compound 로직 
#### input: df_mec_title_all

---

#### case
1. 명사간 -> comp_NNG_NNG
2. VA + EC -> comp_VA_EC
3. VA + EF
4. SN + NR + NNBC
5. SN + NR
6. VCN + EC
7. NNG + XSN + JKB
8. NNG + JKB
9. NNG + XSN
10. MM + NNBC
11. MM + NNG
12. VV + (EP+EF)
13. SN + NNBC
14. VA + ETM
15. XSV + EC
16. SL + NNG
17. ('1', 'SN'), ('년', 'NNBC'), ('차', 'NNG'), 

In [25]:
import os
import pandas as pd

,art_id,token
0,0,"[('erp', 'SL')]"
1,0,"[('개발', 'NNG')]"
2,0,"[('유지', 'NNG'), ('보수', 'NNG')]"
3,0,"[('하', 'VV'), ('고', 'EC'), ('있', 'VX'), ('는데',..."
4,0,"[('프로그램', 'NNG')]"


In [ ]:
# # (CASE1) 복합명사 처리x 기본 형태소 단위 그대로 넣은 경우 비교용

# df_res_basic = pd.DataFrame(columns=['art_id','token','type'])

# for i in range(len(df_mec_title)):
#     for j in range(len(df_mec_title.loc[i][1])):
#         df_res_basic = df_res_basic.append({'art_id':df_mec_title.loc[i][0], 'token':df_mec_title.loc[i][1][j][0], 'type':df_mec_title.loc[i][1][j][1]},ignore_index=True)
               
# print(df_res_basic)

# # df_res_basic.to_csv('df_res_basic.csv', encoding='utf-8-sig', index=False)

# CASE 일괄작업 로직 

#### ( input : df_mec : 저장된거 불러오면 안됨, 무조건 위에서 돌린거 그대로)

In [64]:
# sample
df_mec_sample = df_mec.loc[0: 3000]

df_res_comps = pd.DataFrame(columns=['art_id','token','type'])
# df_res_comps.append({'art_id':, 'token','type'})

for i in range(len(df_mec_sample)):
    # 3개 조합 케이스
    if len(df_mec_sample.loc[i][1]) == 3 :
        # (1) 명사조합처리
        if (df_mec_sample.loc[i][1][0][1] == 'NNG' or df_mec_sample.loc[i][1][0][1] == 'NNP') and (df_mec_sample.loc[i][1][1][1] == 'NNG' or df_mec_sample.loc[i][1][1][1] == 'NNP') and (df_mec_sample.loc[i][1][2][1] == 'NNG' or df_mec_sample.loc[i][1][2][1] == 'NNP'):
                df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0],'token': "".join([df_mec_sample.loc[i][1][0][0], df_mec_sample.loc[i][1][1][0], df_mec_sample.loc[i][1][2][0] ]) ,'type':'comp_Nouns3'},ignore_index=True)
                continue
        
        # (ex) 예외처리
        df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0], 'token':df_mec_sample.loc[i][1][0][0] ,'type':df_mec_sample.loc[i][1][0][1]}, ignore_index=True)
        df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0], 'token':df_mec_sample.loc[i][1][1][0] ,'type':df_mec_sample.loc[i][1][1][1]}, ignore_index=True)
        df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0], 'token':df_mec_sample.loc[i][1][2][0] ,'type':df_mec_sample.loc[i][1][2][1]}, ignore_index=True)
    
    # 2개 조합 케이스
    elif len(df_mec_sample.loc[i][1]) == 2 :
        # (1) 명사조합처리
        if (df_mec_sample.loc[i][1][0][1] == 'NNG' or df_mec_sample.loc[i][1][0][1] == 'NNP') and (df_mec_sample.loc[i][1][1][1] == 'NNG' or df_mec_sample.loc[i][1][1][1] == 'NNP') :
            df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0],'token': "".join([df_mec_sample.loc[i][1][0][0], df_mec_sample.loc[i][1][1][0] ]) ,'type':'comp_Nouns'},ignore_index=True)
            continue
        
        # (ex) 예외처리
        df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0], 'token':df_mec_sample.loc[i][1][0][0] ,'type':df_mec_sample.loc[i][1][0][1]}, ignore_index=True)
        df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0], 'token':df_mec_sample.loc[i][1][1][0] ,'type':df_mec_sample.loc[i][1][1][1]}, ignore_index=True)
    
    # 1개 이하 처리
    elif len(df_mec_sample.loc[i][1]) <= 1 :
        df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0],'token':df_mec_sample.loc[i][1][0][0],'type':df_mec_sample.loc[i][1][0][1]},ignore_index=True)
        continue

    
pd.set_option('display.max_rows', None)
df_res_comps.head(len(df_res_comps))

,art_id,token,type
0,0,erp,SL
1,0,프로그램,NNG
2,0,오류,NNG
3,0,장애조치,comp_Nouns
4,0,방법,NNG
5,1,이런,MM
6,1,PL,SL
7,1,있,VA
8,1,나요,EC
9,2,어떻,VA


In [ ]:
# sample ###############################백업본
df_mec_sample = df_mec.loc[: 100]
df_mec_sample.loc[3][1][0]

df_res_comps = pd.DataFrame(columns=['art_id','token','type'])
# df_res_comps.append({'art_id':, 'token','type'})

for i in range(len(df_mec_sample)):
    if len(df_mec_sample.loc[i][1]) <= 1 :
        df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0],'token':df_mec_sample.loc[i][1][0][0],'type':df_mec_sample.loc[i][1][0][1]},ignore_index=True)
        continue
    for j in range(1,len(df_mec_sample.loc[i][1])):
        if (df_mec_sample.loc[i][1][j-1][1] == 'NNG' or df_mec_sample.loc[i][1][j-1][1] == 'NNP') and (df_mec_sample.loc[i][1][j][1] == 'NNG' or df_mec_sample.loc[i][1][j][1] == 'NNP') :
            df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0],'token': "".join([df_mec_sample.loc[i][1][j-1][0], df_mec_sample.loc[i][1][j][0] ]) ,'type':'comp_Nouns'},ignore_index=True)
            continue
        df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0], 'token':df_mec_sample.loc[i][1][j][0] ,'type':df_mec_sample.loc[i][1][j][1]}, ignore_index=True)

print(df_res_comps)

In [4]:
# (CASE3) mecab 배열상 type 연속성 기반 복합명사화 : NNG,NNP-NNG,NNP 혼재 CASE 
    # 문제: 3개 연달아 등장하는 명사 해결법
    # 문제: 연

df_res_NNG_NNP = pd.DataFrame(columns=['art_id','token','type'])
# df_res_NNG_NNP.append({'art_id':, 'token','type'})

for i in range(len(df_mec)):
    if len(df_mec.loc[i][1]) <= 1 :
        df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token':df_mec.loc[i][1][0][0],'type':df_mec.loc[i][1][0][1]},ignore_index=True)
        continue
    for j in range(1,len(df_mec.loc[i][1])):
        if (df_mec.loc[i][1][j-1][1] == 'NNG' or df_mec.loc[i][1][j-1][1] == 'NNP') and (df_mec.loc[i][1][j][1] == 'NNG' or df_mec.loc[i][1][j][1] == 'NNP') :
            df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][j-1][0], df_mec.loc[i][1][j][0] ]) ,'type':'comp_NNG_NNP'},ignore_index=True)
            continue
        df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][j][0] ,'type':df_mec.loc[i][1][j][1]}, ignore_index=True)


df_res_NNG_NNP.to_csv('df_res_NNG_NNP_test1.csv', encoding='utf-8-sig', index=False)

print(df_res_NNG_NNP)

     art_id token          type
0         0   erp            SL
1         0  프로그램           NNG
2         0    오류           NNG
3         0  장애조치  comp_NNG_NNP
4         0    방법           NNG
...     ...   ...           ...
6578   1000    고수           NNP
6579   1000     님           XSN
6580   1000     들           XSN
6581   1000     주            VX
6582   1000    세요         EP+EF

[6583 rows x 3 columns]
